In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from random import sample
from sklearn import mixture
import itertools


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
trans = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
trans['t_dat'] =  pd.to_datetime(trans['t_dat'], format="%Y-%m-%d")
trans['ym'] = trans.t_dat.dt.to_period('M')
trans.sales_channel_id = trans.sales_channel_id.replace(1,"Online").replace(2,"Offline")
trans.head()

In [8]:
trans.shape

In [9]:
cus = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
ax = cus.age.hist(bins = 40, grid = False)
ax.set_xlabel("age")
ax.set_ylabel("number of customers")

In [17]:
cus[cus.age == 0]

In [126]:
cross_tab_prop = pd.crosstab(index = trans['ym'], columns=trans['sales_channel_id'], normalize = 'index')
cross_tab_prop.plot(figsize=(10, 6), rot = 45)
plt.legend(loc="upper left", ncol=2)
plt.xlabel("year month")
plt.ylabel("Proportion of channels")
# plt.savefig('channels.png')
cross_tab_prop.to_csv("channel_sales.csv")

In [127]:
trans = trans.merge(art[['article_id','prod_name']], on = 'article_id', how = "left")

In [128]:
first_order = pd.value_counts(trans.loc[:,"prod_name"]).iloc[:12].index
fig, ax = plt.subplots(1,1)
sns.countplot(y="prod_name", ax=ax, data=trans, order=first_order)
trans.groupby("prod_name").size().reset_index(name = "counts").sort_values("counts", ascending = False).head(10).to_csv("top_sales.csv")

In [129]:
toplist = trans.groupby("prod_name").size().reset_index(name = "counts").sort_values("counts", ascending = False).head(5).prod_name.to_list()
topnumber = trans.groupby("prod_name").size().reset_index(name = "counts").sort_values("counts", ascending = False).head(5).counts.to_list()
# im = art[art['prod_name'].isin(toplist)][['article_id', 'prod_name' , 'detail_desc']]
# n = sample(range(len(im)),5)
# im = im.iloc[n]
f, ax = plt.subplots(1, 5, figsize=(20,10))
for i in range(5):
    data = art[art['prod_name']==toplist[i]][['article_id', 'prod_name' , 'detail_desc']]
    n = sample(range(len(data)),1)
    data = data.iloc[n]
    desc = data['detail_desc'].iloc[0]
    desc_list = desc.split(' ')
    for j, elem in enumerate(desc_list):
        if j > 0 and j % 5 == 0:
            desc_list[j] = desc_list[j] + '\n'
    desc = ' '.join(desc_list)
    img = mpimg.imread(f'../input/h-and-m-personalized-fashion-recommendations/images/0{str(data.article_id.iloc[0])[:2]}/0{int(data.article_id)}.jpg')
    ax[i].imshow(img)
    ax[i].set_title(f'product name: {data.prod_name.iloc[0]}\n sales number: {topnumber[i]}')
    ax[i].set_xticks([], [])
    ax[i].set_yticks([], [])
    ax[i].grid(False)
    ax[i].set_xlabel(desc, fontsize=10)
plt.show()

In [130]:
art = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
trans = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
trans['t_dat'] =  pd.to_datetime(trans['t_dat'], format="%Y-%m-%d")
trans['ym'] = trans.t_dat.dt.to_period('M')
trans.sales_channel_id = trans.sales_channel_id.replace(1,"Online").replace(2,"Offline")
trans = trans.merge(art[['article_id','index_name']], on = "article_id", how = "left")

In [131]:
abc = trans.groupby("index_name").size().reset_index(name='counts')
abc['volume_prop'] = 100 * abc['counts']  / abc['counts'].sum()
abc = abc.sort_values("volume_prop", ascending=False)
abc = abc.merge(art.groupby("index_name").size().reset_index(name = "group_num"), on = "index_name", how = "left")
abc['group_prop'] = 100 * abc['group_num']  / abc['group_num'].sum()
abc_cumsum = abc[["volume_prop","group_prop"]].cumsum()
abc_cumsum.loc[-1] = [0,0]  
abc_cumsum.index = abc_cumsum.index + 1  
abc_cumsum = abc_cumsum.sort_index()

In [132]:
fig, ax = plt.subplots()
n = abc.index_name
ax.plot(abc_cumsum.group_prop, abc_cumsum.volume_prop, '--bo')
ax.set_xlabel("group proportion (%)")
ax.set_ylabel("volume proportion (%)")
for i, txt in enumerate(n):
    if i > 4: break
    ax.annotate(txt, (abc_cumsum.group_prop[i+1]-10, abc_cumsum.volume_prop[i+1]+3))
plt.axhline(40,color = "black")
plt.axhline(80,color = "black")

In [133]:
abc = abc.merge(trans.groupby(['index_name','ym']).size().reset_index(name = "variance").groupby('index_name').var().apply(np.sqrt),on = "index_name", how = "left")
abc = abc.merge(trans.groupby(['index_name','ym']).size().reset_index(name = "mean").groupby('index_name').mean(),on = "index_name", how = "left")
abc['cv']=abc['variance']/abc['mean']*100

In [134]:
fig, ax = plt.subplots()
n = abc.sort_values("volume_prop", ascending = False).index_name.head(3)
ax.scatter(abc.cv, abc.volume_prop)
ax.set_xlabel("coefficient of variance (\%)")
ax.set_ylabel("volume proportion (\%)")
for i, txt in enumerate(n):
    ax.annotate(txt, (abc.cv[i], abc.volume_prop[i]))
plt.axvline(10,color = "black")
plt.axvline(25,color = "black")
plt.axhline(40,color = "black")

In [135]:
freq = trans[trans.index_name == 'Ladieswear'].groupby(by = "customer_id").size().reset_index(name='counts')
mon = trans[trans.index_name == 'Ladieswear'].groupby(by = "customer_id").price.sum().reset_index()
rec = trans[trans.index_name == 'Ladieswear'].groupby(by = "customer_id").t_dat.max().reset_index()
fm = pd.merge(freq,mon,on = "customer_id")
rfm = pd.merge(rec,fm,on = "customer_id")
rfm['date'] = rfm['t_dat'].values.astype(np.int64) // 10 ** 9

In [136]:
from scipy.stats import boxcox
_, lam = boxcox(rfm['price'])
print(lam)
_, lam = boxcox(rfm['counts'])
print(lam)

In [137]:
rfm['logprice'] = np.log(rfm['price'])
rfm['logcounts'] = np.log(rfm['counts'])
rfm['date'] = (rfm['date']-rfm.date.min())/(rfm.date.max()-rfm.date.min())*10
X = rfm[["logcounts","logprice","date"]]

In [138]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 13)
cv_types = ["spherical", "tied", "diag", "full"]
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(
            n_components=n_components, covariance_type=cv_type
        )
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(["navy", "turquoise", "cornflowerblue", "darkorange"])
clf = best_gmm
bars = []

# Plot the BIC scores
plt.figure(figsize=(8, 6))
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + 0.2 * (i - 2)
    bars.append(
        plt.bar(
            xpos,
            bic[i * len(n_components_range) : (i + 1) * len(n_components_range)],
            width=0.2,
            color=color,
        )
    )
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - 0.01 * bic.max(), bic.max()])
plt.title("BIC score per model")
xpos = (
    np.mod(bic.argmin(), len(n_components_range))
    + 0.65
    + 0.2 * np.floor(bic.argmin() / len(n_components_range))
)
plt.text(xpos, bic.min() * 0.97 + 0.03 * bic.max(), "*", fontsize=14)
spl.set_xlabel("Number of components")
spl.legend([b[0] for b in bars], cv_types)

In [139]:
gmm = mixture.GaussianMixture(
            n_components=12, covariance_type="diag"
        )
gmm.fit(X)
rfm['gmm'] = gmm.predict(X)
plt.scatter(rfm["counts"], rfm["price"], c=rfm['gmm'], cmap=plt.cm.Set1)

In [140]:
plt.scatter(rfm["date"], rfm["price"], c=rfm['gmm'], cmap=plt.cm.Set1)

In [141]:
plt.scatter(rfm["counts"], rfm["date"], c=rfm['gmm'], cmap=plt.cm.Set1)

In [142]:
rfm.groupby(by = "gmm").size()

In [143]:
g = sns.boxplot(x = "gmm", y = "price", data = rfm)
g.set_yscale("log")

In [144]:
g = sns.boxplot(x = "gmm", y = "counts", data = rfm)
g.set_yscale("log")

In [145]:
sns.boxplot(x = "gmm", y = "date", data = rfm)

In [146]:
trans = trans.merge(rfm[['customer_id','gmm']], on = 'customer_id', how = 'left')
trans = trans[trans.index_name == "Ladieswear"]

In [147]:
trans.gmm = np.where(trans['gmm'] == 11, "Target", 'Others')
rfm['Group']  = np.where(rfm['gmm'] == 11, "Target", 'Others')

In [148]:
my_pal = {"Target": "red", "Others": "darkgray"}
g = sns.boxplot(x = 'Group', y = 'price', data = rfm,palette=my_pal)
g.set_yscale("log")
g.set(ylabel = 'montery')
g.figure.savefig("montery.png")

In [149]:
g = sns.boxplot(x = 'Group', y = 'counts', data = rfm,palette=my_pal)
g.set_yscale("log")
g.set(ylabel = 'frequency')
g.figure.savefig("frequency.png")

In [150]:
g = sns.boxplot(x = 'Group', y = 'date', data = rfm ,palette=my_pal)
g.set(ylabel = 'recency')
g.figure.savefig("recency.png")

In [151]:
trans[trans.gmm == 'Target'].price.sum() / trans.price.sum()

In [152]:
trans.sales_channel_id = trans.sales_channel_id.replace(1,"Online").replace(2,"Offline")

In [153]:
del abc
del abc_cumsum

In [154]:
cross_tab_prop = pd.crosstab(index = trans['gmm'], columns=trans['sales_channel_id'], normalize="index")
cross_tab_prop.plot(kind='barh',stacked=True, figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("Proportion of channels")
plt.ylabel("Group")
for n, x in enumerate([*cross_tab_prop.index.values]):
    for (proportion,  y_loc) in zip(cross_tab_prop.loc[x],
                                          cross_tab_prop.loc[x].cumsum()):
                
        plt.text(x=(y_loc - proportion) + (proportion / 2)-0.04,
                 y=n,
                 s=f'{np.round(proportion * 100, 1)}', 
                 color="white",
                 fontsize=18,
                 fontweight="bold")
plt.show()


In [155]:
cross_tab_prop.to_csv("onoffline.csv")

In [156]:
trans = trans.merge(art[['article_id',
       'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'section_name', 'garment_group_name']], on = 'article_id', how = 'left')

* Target: Lucy blouse

In [157]:
cus = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
cus = trans[['customer_id','gmm']].merge(cus, on = 'customer_id', how = 'left')

In [158]:
cross_tab_prop = pd.crosstab(index = trans['product_group_name'], columns=trans['gmm'])
cross_tab_prop.Others = cross_tab_prop.Others / 1.5
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper right", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
cross_tab_prop.to_csv("product_group_name.csv")

In [159]:
cross_tab_prop = pd.crosstab(index = trans['graphical_appearance_name'], columns=trans['gmm'])
cross_tab_prop.Others = cross_tab_prop.Others / 1.5
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.axvline(0.66, color = 'black')
plt.axvline(0.5, color = 'black')
plt.show()
del cross_tab_prop

In [160]:
cross_tab_prop = pd.crosstab(index = trans['section_name'], columns=trans['gmm'])
cross_tab_prop.Others = cross_tab_prop.Others / 1.5
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper right", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.axvline(0.66, color = 'black')
plt.axvline(0.5, color = 'black')
plt.show()
cross_tab_prop.to_csv("section_name.csv")

In [161]:
art.drop('detail_desc',inplace = True, axis = 1)

In [162]:
art[art.section_name == 'Womens Trend'].columns

In [163]:
art[art.section_name == 'Womens Trend'].department_name.unique()

In [164]:
cross_tab_prop = pd.crosstab(index = trans['garment_group_name'], columns=trans['gmm'])
cross_tab_prop.Others = cross_tab_prop.Others / 1.5
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper right", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [165]:
cross_tab_prop = pd.crosstab(index = trans[trans.product_group_name=='Garment Full body']['graphical_appearance_name'], columns=trans[trans.product_group_name=='Garment Full body']['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [166]:
cross_tab_prop = pd.crosstab(index = trans[trans.product_group_name=='Garment Full body']['section_name'], columns=trans[trans.product_group_name=='Garment Full body']['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [167]:
cross_tab_prop = pd.crosstab(index = trans[trans.product_group_name=='Garment Full body']['garment_group_name'], columns=trans[trans.product_group_name=='Garment Full body']['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [168]:
cross_tab_prop = pd.crosstab(index = trans[trans.section_name=='H&M+']['graphical_appearance_name'], columns=trans[trans.section_name=='H&M+']['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [169]:
cross_tab_prop = pd.crosstab(index = trans[trans.section_name.isin(['Womens Trend'])]['graphical_appearance_name'], columns=trans[trans.section_name.isin(['Womens Trend'])]['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [170]:
cross_tab_prop = pd.crosstab(index = trans[trans.section_name=='H&M+']['garment_group_name'], columns=trans[trans.section_name=='H&M+']['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [171]:
cross_tab_prop = pd.crosstab(index = trans[trans.section_name.isin(['Womens Trend'])]['garment_group_name'], columns=trans[trans.section_name.isin(['Womens Trend'])]['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
cross_tab_prop.to_csv("Womens_Trend.csv")

In [172]:
cross_tab_prop = pd.crosstab(index = trans[trans.section_name=='H&M+']['product_group_name'], columns=trans[trans.section_name=='H&M+']['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [173]:
cross_tab_prop = pd.crosstab(index = trans[trans.section_name.isin(['Womens Trend'])]['product_group_name'], columns=trans[trans.section_name.isin(['Womens Trend'])]['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [174]:
cross_tab_prop = pd.crosstab(index = trans[trans.garment_group_name.isin(['Blouses','Dresses Ladies'])]['product_group_name'], columns=trans[trans.garment_group_name.isin(['Blouses','Dresses Ladies'])]['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [175]:
cross_tab_prop = pd.crosstab(index = trans[trans.garment_group_name.isin(['Blouses','Dresses Ladies'])]['graphical_appearance_name'], columns=trans[trans.garment_group_name.isin(['Blouses','Dresses Ladies'])]['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [176]:
cross_tab_prop = pd.crosstab(index = trans[trans.garment_group_name.isin(['Blouses','Dresses Ladies'])]['section_name'], columns=trans[trans.garment_group_name.isin(['Blouses','Dresses Ladies'])]['gmm'])
cross_tab_prop.plot(kind='barh', figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.show()
del cross_tab_prop

In [177]:
cus = cus.drop(["postal_code"], axis = 1)
cus["fashion_news_frequency"] = cus["fashion_news_frequency"].replace("NONE",np.nan)
cus["fashion_news_frequency"] = cus["fashion_news_frequency"].replace("None",np.nan)
cus = cus.fillna(0)
cus.drop_duplicates(inplace = True)

In [178]:
cross_tab_prop = pd.crosstab(index = cus['gmm'], columns=cus['FN'], normalize = 'index')
cross_tab_prop.plot(kind='barh', stacked = True, figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.axvline(0.5, color = "black")
plt.show()
del cross_tab_prop

In [179]:
cross_tab_prop = pd.crosstab(index = cus['gmm'], columns=cus['Active'], normalize = 'index')
cross_tab_prop.plot(kind='barh', stacked = True, figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.axvline(0.5, color = "black")
plt.show()
del cross_tab_prop

In [180]:
cross_tab_prop = pd.crosstab(index = cus['gmm'], columns=cus['club_member_status'], normalize = 'index')
cross_tab_prop.plot(kind='barh', stacked = True, figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.axvline(0.5, color = "black")
plt.show()
del cross_tab_prop

In [181]:
cross_tab_prop = pd.crosstab(index = cus['gmm'], columns=cus['fashion_news_frequency'], normalize = 'index')
cross_tab_prop.plot(kind='barh', stacked = True, figsize=(10, 6))
plt.legend(loc="upper left", ncol=2)
plt.xlabel("product_group_name")
plt.ylabel("Group")
plt.axvline(0.5, color = "black")
plt.show()
del cross_tab_prop


In [185]:
cus[cus.age<15].gmm == "Target"

In [190]:
plt.hist(cus[(cus.gmm == 'Target')&(cus.age>0)].age, bins = 40, alpha=0.5, label='Target', density = True, color = "red")
plt.hist(cus[(cus.gmm == 'Others')&(cus.age>0)].age, bins = 40, alpha=0.5, label='Others', density = True, color = "gray")
plt.legend(loc='upper right')
plt.xlabel("age")
plt.ylabel("density")
plt.show()